# Focus Zone

> Configuration for focusable containers with navigable items.

In [ ]:
#| default_exp core.focus_zone

In [ ]:
#| export
from __future__ import annotations
from dataclasses import dataclass, field
from typing import Optional, Union

from cjm_fasthtml_keyboard_navigation.core.navigation import (
    NavigationPattern,
    LinearVertical
)

from cjm_fasthtml_tailwind.utilities.effects import ring, inset_ring
from cjm_fasthtml_daisyui.utilities.semantic_colors import ring_dui

## FocusZone

A container that can receive focus and contains navigable items. Each zone tracks its own focus state and can have independent navigation patterns.

In [ ]:
#| export
@dataclass
class FocusZone:
    """A focusable container with navigable items."""
    
    # Identity
    id: str  # HTML element ID of the container

    # Item selection
    item_selector: Optional[str] = None  # CSS selector for items (None = scroll only)

    # Navigation
    navigation: Union[NavigationPattern, LinearVertical] = field(
        default_factory=LinearVertical
    )  # navigation pattern for this zone
    navigation_throttle_ms: int = 0  # minimum ms between navigation events (0 = no throttle)

    # Visual feedback - item focus
    item_focus_classes: tuple[str, ...] = (str(ring(2)), str(ring_dui.primary))  # CSS classes for focused item
    item_focus_attribute: str = "data-focused"  # attribute set to "true" on focused item

    # Visual feedback - zone focus (when zone is active)
    zone_focus_classes: tuple[str, ...] = (
        str(ring(2)), str(ring_dui.primary), str(inset_ring(2))
    )  # CSS classes for active zone container

    # Data extraction (for HTMX hidden inputs)
    data_attributes: tuple[str, ...] = ()  # data attributes to extract from focused item

    # Callbacks (JS function names)
    on_focus_change: Optional[str] = None  # called when focused item changes
    on_navigate: Optional[str] = None  # called on any navigation (for side effects like audition)
    on_zone_enter: Optional[str] = None  # called when zone becomes active
    on_zone_leave: Optional[str] = None  # called when zone loses focus

    # Scroll behavior
    scroll_behavior: str = "smooth"  # "smooth" or "auto"
    scroll_block: str = "nearest"  # "start", "center", "end", "nearest"

    # HTMX integration
    hidden_input_prefix: str = ""  # prefix for auto-generated hidden input IDs

    # Initial state
    initial_index: int = 0  # initial focused item index

    def has_items(self) -> bool: # True if zone has selectable items
        """Check if zone has selectable items."""
        return self.item_selector is not None

    def get_hidden_input_id(
        self,
        attr: str  # the data attribute name
    ) -> str:      # the hidden input element ID
        """Get the hidden input ID for a data attribute."""
        prefix = self.hidden_input_prefix or self.id
        return f"{prefix}-{attr}"

    def to_js_config(self) -> dict: # JavaScript-compatible configuration
        """Convert to JavaScript configuration object."""
        return {
            "id": self.id,
            "itemSelector": self.item_selector,
            "navigationPattern": self.navigation.name,
            "navigationThrottleMs": self.navigation_throttle_ms,
            "itemFocusClasses": list(self.item_focus_classes),
            "itemFocusAttribute": self.item_focus_attribute,
            "zoneFocusClasses": list(self.zone_focus_classes),
            "dataAttributes": list(self.data_attributes),
            "hiddenInputPrefix": self.hidden_input_prefix or self.id,
            "onFocusChange": self.on_focus_change,
            "onNavigate": self.on_navigate,
            "onZoneEnter": self.on_zone_enter,
            "onZoneLeave": self.on_zone_leave,
            "scrollBehavior": self.scroll_behavior,
            "scrollBlock": self.scroll_block,
            "initialIndex": self.initial_index,
        }

In [ ]:
# Test FocusZone basics
zone = FocusZone(
    id="source-browser",
    item_selector="tr[data-selectable='true']",
    data_attributes=("job-id", "plugin-name"),
    on_focus_change="updatePreview"
)

assert zone.id == "source-browser"
assert zone.has_items() == True
assert zone.get_hidden_input_id("job-id") == "source-browser-job-id"

# Test to_js_config
config = zone.to_js_config()
assert config["id"] == "source-browser"
assert config["itemSelector"] == "tr[data-selectable='true']"
assert config["onFocusChange"] == "updatePreview"

In [ ]:
# Test scroll-only zone
from cjm_fasthtml_keyboard_navigation.core.navigation import ScrollOnly

preview_zone = FocusZone(
    id="preview-panel",
    item_selector=None,
    navigation=ScrollOnly()
)

assert preview_zone.has_items() == False
assert preview_zone.navigation.name == "scroll_only"

In [ ]:
# Test with custom hidden input prefix
zone_with_prefix = FocusZone(
    id="my-zone",
    hidden_input_prefix="kb",
    data_attributes=("file-path",)
)

assert zone_with_prefix.get_hidden_input_id("file-path") == "kb-file-path"

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()